In [1]:
import pandas as pd
import numpy as np
import keras
import string
from nltk import SnowballStemmer

Using TensorFlow backend.


In [2]:
import numpy as np
np.zeros((2,3))

array([[0., 0., 0.],
       [0., 0., 0.]])

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\彭伟林\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
train = pd.read_csv('./train.csv')

In [5]:
train.head()

,label,text
0,2,Gas by my house hit $3.39!!!! I'm going to Cha...
1,0,Iranian general says Israel's Iron Dome can't ...
2,2,with J Davlar 11th. Main rivals are team Polan...
3,0,"Talking about ACT's &amp;&amp; SAT's, deciding..."
4,0,"They may have a SuperBowl in Dallas, but Dalla..."


In [6]:
x_train = train['text']
y_train = train['label']

In [7]:
import re
from nltk.corpus import stopwords
import nltk.stem

stopwords_eng = set(stopwords.words("english"))
'''
def clean_text(text):
    text = text.lower() # convert to lowercase
    text = re.sub("[^a-z]", " ", text)
    # words = [word for word in text.split() if word not in stopwords_eng]
    words = [word for word in text.split()]
    text = " ".join(words)
    return text
'''
def clean_text(text):
    #1. Remove　ＨTML
    # text = BeautifulSoup(raw_review).get_text()
    ## Remove puncuation
    text = text.translate(string.punctuation)
    ## Convert words to lower case and split them
    text = text.lower().split()
    # Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    ## Stemming
    text = text.split()
    stemmer = nltk.stem.SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    return text

x_train = x_train.map(clean_text)
x_train.head()

0    gas hous hit 3 39 ! ! ! ! i am go chapel hill sat
1    iranian general say israel iron dome ca not de...
2    davlar 11th main rival team poland hope make s...
3    talk act amp; amp; sat decid want colleg appli...
4    may superbowl dalla dalla ai not win superbowl...
Name: text, dtype: object

In [8]:
from keras.preprocessing.text import Tokenizer
num_words = 5000
tokenizer = Tokenizer(num_words=num_words)

tokenizer.fit_on_texts(x_train)

x_train_seq = pd.Series(tokenizer.texts_to_sequences(x_train))
x_train_seq.head()

0    [2312, 132, 263, 87, 2860, 5, 18, 9, 2313, 706...
1    [1658, 59, 1086, 1778, 3629, 71, 48, 461, 264,...
2    [592, 899, 3198, 85, 1933, 49, 29, 1212, 97, 1...
3    [122, 480, 16, 16, 60, 616, 35, 435, 1659, 435...
4        [7, 1213, 506, 506, 1152, 48, 58, 1213, 1299]
dtype: object

In [9]:
x_train_len = x_train_seq.map(lambda ls: len(ls))
x_train_len.describe()

count    10026.000000
mean        11.095651
std          3.864452
min          0.000000
25%          8.000000
50%         11.000000
75%         14.000000
max         61.000000
dtype: float64

In [10]:
from keras.preprocessing.sequence import pad_sequences

x_train_pad = pad_sequences(x_train_seq, maxlen=512)

In [11]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, GlobalMaxPool1D, Dropout, Activation

model = 0
model = Sequential()

model.add(Embedding(input_dim=num_words, output_dim=64))
model.add(LSTM(256))
#model.add(GlobalMaxPool1D())
#model.add(Dense(16, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 64)          320000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               328704    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 771       
Total params: 649,475
Trainable params: 649,475
Non-trainable params: 0
_________________________________________________________________


In [12]:
y_train = keras.utils.to_categorical(y_train,num_classes=3)

In [13]:
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])

batch_size = 32
epochs = 10
validation_split = 0.1
model.fit(x=x_train_pad, y=y_train, batch_size=batch_size, epochs=epochs, validation_split=validation_split)

Instructions for updating:
Use tf.cast instead.
Train on 9023 samples, validate on 1003 samples
Epoch 1/10
9023/9023 [==============================] - 248s 27ms/step - loss: 0.5377 - acc: 0.7284 - val_loss: 0.5363 - val_acc: 0.7454
Epoch 2/10
9023/9023 [==============================] - 244s 27ms/step - loss: 0.3854 - acc: 0.8280 - val_loss: 0.5211 - val_acc: 0.7591
Epoch 3/10
9023/9023 [==============================] - 244s 27ms/step - loss: 0.3356 - acc: 0.8630 - val_loss: 0.5460 - val_acc: 0.7594
Epoch 4/10
9023/9023 [==============================] - 243s 27ms/step - loss: 0.2528 - acc: 0.9005 - val_loss: 0.6218 - val_acc: 0.7501
Epoch 5/10
9023/9023 [==============================] - 244s 27ms/step - loss: 0.2040 - acc: 0.9223 - val_loss: 0.7311 - val_acc: 0.7461
Epoch 6/10
9023/9023 [==============================] - 244s 27ms/step - loss: 0.1602 - acc: 0.9420 - val_loss: 0.8284 - val_acc: 0.7338
Epoch 7/10
9023/9023 [==============================] - 244s 27ms/step - loss: 0.1

In [14]:
test = pd.read_csv("./sentence-sentiment-analysis/test.tsv", delimiter="\t", quoting=3, header=0)
test.head()

,1,id,3,text
0,NaN,1,unknwn,dec 21st 2012 will be know not as the end of t...
1,NaN,2,unknwn,Yar he quite clever but aft many guesses lor. ...
2,NaN,3,unknwn,Yeah we have Thin Lizzy here I HATE the inform...
3,NaN,4,unknwn,"MT @LccSy #Syria, Deir Ezzor: Ali Bashar al-th..."
4,NaN,5,unknwn,"@MacMiller hate my life, because i can't see y..."


In [15]:
test = test.text

In [16]:
x_test = test.map(clean_text)
x_test.head()

0    dec 21st 2012 know end world babi boom ! 2012shit
1    yar quit clever aft mani guess lor got ask bri...
2                      yeah thin lizzi hate informerci
3    lccsi syria deir ezzor : ali bashar al - theeb...
4    macmil hate life ca not see roskild festiv sat...
Name: text, dtype: object

In [17]:
x_test_seq = tokenizer.texts_to_sequences(x_test)
x_test_pad = pad_sequences(x_test_seq, maxlen=512)

In [18]:
pred = model.predict(x=x_test_pad)

In [19]:
v = np.argmax(pred, axis=1)

In [20]:
v

array([2, 1, 0, ..., 0, 1, 2], dtype=int64)

In [21]:
p = []
for i in v:
    if i == 0:
        p.append('negative')
    elif i == 1:
        p.append('neutral')
    else:
        p.append('positive')

In [22]:
df = pd.DataFrame({'polarity':p})
df.index += 1
df.to_csv('sub.csv', index = True, index_label = 'id')